In [1]:
N_JOBS = 7
N_MACHINES = 4

In [2]:
import random

def generateData(seed=0):
    PROC = [ [ 0 for m in range(N_MACHINES) ] for j in range(N_JOBS) ]
    random.seed(seed)
    for j in range(N_JOBS):
        for m in range(N_MACHINES):
            PROC[j][m] = random.randint(1,9)
    return PROC

In [3]:
PROC = generateData(4453)

In [4]:
PROC

[[2, 5, 2, 1],
 [8, 8, 6, 9],
 [5, 6, 3, 8],
 [3, 1, 5, 4],
 [7, 8, 3, 5],
 [7, 6, 1, 9],
 [9, 8, 3, 5]]

## Greedy Algorithm

In [5]:
import random


def val(x):
    return int(x)

def greedy_algorithm(PROC):
    N_JOBS = len(PROC)
    N_MACHINES = len(PROC[0])
    
    # Initialize scheduling variables
    Idle = [[0 for _ in range(N_MACHINES)] for _ in range(N_JOBS)]
    Start = [[0 for _ in range(N_MACHINES)] for _ in range(N_JOBS)]
    Stop = [[0 for _ in range(N_MACHINES)] for _ in range(N_JOBS)]
    
    # Initialize machine availability times
    machine_times = [0] * N_MACHINES
    
    # Schedule each job
    for j in range(N_JOBS):
        for m in range(N_MACHINES):
            if m == 0:
                Start[j][m] = machine_times[m]
            else:
                Start[j][m] = max(machine_times[m], Stop[j][m-1])
            
            Idle[j][m] = Start[j][m] - (machine_times[m] if m == 0 else Stop[j][m-1])
            Stop[j][m] = Start[j][m] + PROC[j][m]
            machine_times[m] = Stop[j][m]
    
    return Idle, Start, Stop, PROC

In [6]:
def schedule(N_JOBS, N_MACHINES, Idle, Start, Stop, PROC):
    row = ""
    row += '|---------|'
    for m in range(N_MACHINES):
        row += f"------------|" \
                f"------------|"
    row += '\n'
    row += '|         |'
    for m in range(N_MACHINES):
        row += f"            |" \
                f" Machine: {m:1d} |"
    row += '\n'
    for j in range(N_JOBS):
        row += '|---------|'
        for m in range(N_MACHINES):
            row += f"------------|" \
                   f"------------|"
        row += '\n'
        row += '|         |'
        for m in range(N_MACHINES):
            row += f"       {' ':4s} | " \
                   f"Idle: {val(Idle[j][m]):4d} |"
        row += '\n'
        row += '|---------|'
        for m in range(N_MACHINES):
            row += f"------------|" \
                   f"------------|"
        row += '\n'
        row += '|         |'
        for m in range(N_MACHINES):
            row += f"       {' ':4s} | " \
                   f"Start:{val(Start[j][m]):4d} |"  
        row += '\n' 
        row += f'| Job: {j:2d} |'
        for m in range(N_MACHINES):
            row += f" Wait: {val(Idle[j][m]):4d} | " \
                   f"Proc: {PROC[j][m]:4d} |"
        row += '\n'
        row += '|         |'
        for m in range(N_MACHINES):
            row += f"       {' ':4s} | " \
                   f"Stop: {val(Stop[j][m]):4d} |"  
        row += '\n' 
    row += '|---------|'
    for m in range(N_MACHINES):
        row += f"------------|" \
                f"------------|"
    row += '\n'
    return row

Idle, Start, Stop, PROC = greedy_algorithm(PROC)
print("Greedy Algorithm Result:")
print("Job Sequence and Detailed Schedule:")
print(schedule(7, 4, Idle, Start, Stop, PROC))

Greedy Algorithm Result:
Job Sequence and Detailed Schedule:
|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Machine: 0 |            | Machine: 1 |            | Machine: 2 |            | Machine: 3 |
|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Idle:    0 |            | Idle:    0 |            | Idle:    0 |            | Idle:    0 |
|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Start:   0 |            | Start:   2 |            | Start:   7 |            | Start:   9 |
| Job:  0 | Wait:    0 | Proc:    2 | Wait:    0 | Proc:    5 | Wait:    0 | Proc:    2 | Wait:    0 | Proc:    1 |
|         |            | Stop:    2 |            | Stop:    7 |            | Stop:    9 |            | Stop:   10 |
|---------|

In [7]:
stop_times = [
    [2, 7, 9, 10],
    [10, 18, 24, 33],
    [15, 24, 27, 41],
    [18, 25, 32, 45],
    [25, 33, 36, 50],
    [32, 39, 40, 59],
    [41, 49, 52, 64]
]

def calculate_overall_processing_time(stop_times):
    overall_processing_time = max(max(times) for times in stop_times)
    return overall_processing_time

overall_processing_time = calculate_overall_processing_time(stop_times)

print("Overall Processing Time:", overall_processing_time)


Overall Processing Time: 64


## Stochastic Opimisation algorithm (Simulated Annealing algorithm)

In [8]:
import random
import math

def calculate_makespan(job_order, PROC, N_MACHINES):
    # Initialize completion times for machines
    machine_times = [0] * N_MACHINES
    
    for job in job_order:
        # Start the first job at the completion time of the previous job on the first machine
        current_start_time = machine_times[0]
        for machine in range(N_MACHINES):
            if machine == 0:
                # Job starts after the previous job finished on this machine
                machine_times[machine] += PROC[job][machine]
            else:
                # Job starts after both the previous job on this machine and this job on the previous machine
                machine_times[machine] = max(machine_times[machine], machine_times[machine - 1]) + PROC[job][machine]
    return machine_times[-1]

def initial_solution(N_JOBS):
    return list(range(N_JOBS))

def get_neighbor(solution):
    # Swap two random elements
    a, b = random.sample(range(len(solution)), 2)
    solution[a], solution[b] = solution[b], solution[a]
    return solution

def simulated_annealing(PROC, N_JOBS, N_MACHINES, temp=1000, cooling_rate=0.99, stopping_temp=1):
    current_solution = initial_solution(N_JOBS)
    current_cost = calculate_makespan(current_solution, PROC, N_MACHINES)
    while temp > stopping_temp:
        new_solution = get_neighbor(current_solution[:])
        new_cost = calculate_makespan(new_solution, PROC, N_MACHINES)
        if new_cost < current_cost or random.uniform(0, 1) < math.exp((current_cost - new_cost) / temp):
            current_solution, current_cost = new_solution, new_cost
        temp *= cooling_rate
    return current_solution, current_cost

# Set a seed for reproducibility
random.seed(4453)
best_solution, best_cost = simulated_annealing(PROC, N_JOBS, N_MACHINES)
print("Best solution:", best_solution)
print("Best cost (makespan):", best_cost)


Best solution: [5, 4, 1, 2, 3, 6, 0]
Best cost (makespan): 63


In [9]:
def calculate_schedule(job_order, PROC, N_JOBS, N_MACHINES):
    # Arrays to store Idle times, Start times, and Stop times for each job on each machine
    Idle = [[0] * N_MACHINES for _ in range(N_JOBS)]
    Start = [[0] * N_MACHINES for _ in range(N_JOBS)]
    Stop = [[0] * N_MACHINES for _ in range(N_JOBS)]

    #Track the end times of the last job on each machine
    machine_end_times = [0] * N_MACHINES

    #Process each job as per the job_order
    for job in job_order:
        for m in range(N_MACHINES):
            if m == 0:
                # First machine: Job starts as soon as the previous job finished
                Start[job][m] = machine_end_times[m]
            else:
                # Subsequent machines: Job starts after both the previous machine has started this job and the same machine has finished the previous job
                Start[job][m] = max(Stop[job][m-1], machine_end_times[m])

            # Processing time added to the start time to determine when it stops
            Stop[job][m] = Start[job][m] + PROC[job][m]

            # Calculate Idle time
            if m == 0:
                Idle[job][m] = 0  # First machine has no idle time for the first job
            else:
                Idle[job][m] = Start[job][m] - machine_end_times[m]

            # Update the end time for this machine
            machine_end_times[m] = Stop[job][m]

    return Idle, Start, Stop

# Use Simulated Annealing to get the best job order
best_solution, best_cost = simulated_annealing(PROC, N_JOBS, N_MACHINES)

# Calculate the schedule arrays
Idle, Start, Stop = calculate_schedule(best_solution, PROC, N_JOBS, N_MACHINES)

# Use the schedule function to display the detailed schedule
print("Simulated Annealing Result:")
print("Job Sequence and Detailed Schedule:")
print(schedule(N_JOBS, N_MACHINES, Idle, Start, Stop, PROC))


Simulated Annealing Result:
Job Sequence and Detailed Schedule:
|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Machine: 0 |            | Machine: 1 |            | Machine: 2 |            | Machine: 3 |
|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Idle:    0 |            | Idle:    0 |            | Idle:    2 |            | Idle:    0 |
|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Start:  15 |            | Start:  24 |            | Start:  29 |            | Start:  36 |
| Job:  0 | Wait:    0 | Proc:    2 | Wait:    0 | Proc:    5 | Wait:    2 | Proc:    2 | Wait:    0 | Proc:    1 |
|         |            | Stop:   17 |            | Stop:   29 |            | Stop:   31 |            | Stop:   37 |
|-------

In [10]:
def calculate_overall_processing_time(Stop):
    max_stop_time = 0
    for job_stops in Stop:
        job_max = max(job_stops)
        if job_max > max_stop_time:
            max_stop_time = job_max
    return max_stop_time

# Example Stop matrix from your provided schedule:
Stop = [
    [17, 29, 31, 37],
    [8, 16, 22, 31],
    [41, 50, 53, 63],
    [20, 30, 36, 41],
    [15, 24, 27, 36],
    [27, 36, 37, 50],
    [36, 44, 47, 55]
]

# Calculate the overall processing time
overall_processing_time = calculate_overall_processing_time(Stop)
print("Overall Processing Time:", overall_processing_time)


Overall Processing Time: 63
